In [322]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import MySQLdb
from MySQLdb.constants import FIELD_TYPE
from datetime import datetime
import calendar
import holidays
import random
from sklearn import cross_validation
from sklearn import datasets
from sklearn import preprocessing



# start_date = raw_input("Start Date in form: YYYY-MM-DD")
# end_date = raw_input("End Date in form YYYY-MM-DD")

#Acquire DAM SPP for the different settlementpoints for a specific date range
class Query_DAM_by_SP(object):
    #initialization of MySQLdb, shared by all instances of Query_DAM_by_SP
    my_conv = { FIELD_TYPE.DECIMAL: float }
    db=MySQLdb.connect(host="localhost",db="ercot_data",read_default_file="/etc/mysql/my.cnf",conv=my_conv)
    c=db.cursor()
    c.execute("""SELECT DISTINCT settlement_point FROM DAM_prices_by_SPP""")
    r = list(c.fetchall())
    settlement_points = []
    for settlement_point in r:
        if settlement_point[0] == "\"Settlement Point\"":
            continue
        settlement_points.append(settlement_point[0])
       
    def __init__(self):
        pass
         
    #query for all prices for all load zones and hubs for specified date range
    def query(self,sd,ed):
        self.start_date = sd
        self.end_date = ed
        self.result_dict = {}
        for (idx,val) in enumerate(settlement_points):
            c.execute("""SELECT delivery_date,hour_ending,spp 
                FROM DAM_prices_by_SPP 
                WHERE settlement_point = "%s" 
                AND delivery_date > "%s" 
                AND delivery_date < "%s" 
                ORDER BY delivery_date,hour_ending""" % (val,self.start_date,self.end_date))
            self.result = list(c.fetchall())
            self.result_dict[val] = self.result
        self.spp_dict = {}
        self.dts = []
        self.count = 0
        for bus_name,result in self.result_dict.iteritems():
            self.spps = []
            for (date,time,spp) in self.result:
                time = str(int(time.split(":")[0])-1)
                dt = datetime.strptime(date + " " + time, "%Y-%m-%d %H")
                self.spps.append(float(spp))
                if self.count == 0:
                    self.dts.append(dt)
            self.count = self.count + 1
            self.spp_dict[bus_name] = self.spps
        self.string_dts = [dt.strftime("%Y-%m-%d %H") for dt in self.dts]
        self.df = pd.DataFrame(data = self.spp_dict, index = self.string_dts)
        
    def construct_feature_vector_matrix(self,lzhub,model_type):
        self.dflzhub = self.df[lzhub]
        self.features = []
        if model_type == "A":
            for dt,price in self.dflzhub.iteritems():
                pred_hour_index = self.dflzhub.index.get_loc(dt)
                if pred_hour_index - 7*24 >= 0:
                    self.features.append([work_day_or_holiday(string_to_date(dt)), \
                                          string_to_date(dt).hour, \
                                          string_to_date(dt).weekday()]\
                                          + self.dflzhub.iloc[pred_hour_index - 2*24:pred_hour_index - 1*24].tolist()\
                                          + self.dflzhub.iloc[pred_hour_index - 7*24:pred_hour_index - 6*24].tolist())
            self.feature_labels = ['Holiday','Hour','Day']\
                                + [('P(h-%s)' % str(i+1)) for i in range(24,48)][::-1]\
                                + [('P(h-%s)' % str(i+1)) for i in range(144,168)][::-1]
                
            self.idx_wout_1st_week = list(self.dflzhub.index.values)[7*24:]
            self.features_df = pd.DataFrame(data = self.features,\
                                            index = self.idx_wout_1st_week,\
                                            columns = self.feature_labels)
            self.features_df = encode_onehot(self.features_df,'Day')
            return self.features_df.join(self.dflzhub,how='left')
            
    def cross_validation_split(self):
        pass
    
    def plot(self):
        #plot the data for a date range
        for bus_name,price in self.spp_dict.iteritems():
            if bus_name != "date_time":
                plt.plot(self.dts,price,label="%s" % bus_name)
        plt.title("SPP by LZ and HUB for %s" % start_date.split("-")[0])
        plt.xlabel("Date-Time")
        plt.ylabel("SPP")
        plt.legend()
        plt.show()

    def get_settlement_points(self):
        return settlement_points
    
def string_to_date(string_date):
    return datetime.strptime(string_date,"%Y-%m-%d %H")

def date_to_string(date):
    return date.strftime("%Y-%m-%d %H")

def weekday_of_date(date):
    return calendar.day_name[date.weekday()]

def work_day_or_holiday(date):
    us_holidays = holidays.UnitedStates()
    if date in us_holidays or weekday_of_date(date) == "Sunday" or weekday_of_date(date) == "Saturday":
        return int(1)
    else: return int(0)
    
def encode_onehot(df, cols):
    enc = preprocessing.OneHotEncoder()
    index = df[cols].index
    data = enc.fit_transform(df[cols].reshape(-1,1)).toarray()
    one_hot_df = pd.DataFrame(data = data,\
                              index = index,\
                              columns = [cols + '%s' % i for i in range(data.shape[1])])
    del df[cols]
    return df.join(one_hot_df,how='inner')

In [323]:
qdsp = Query_DAM_by_SP()
qdsp.query("2012-01-01","2012-12-31")
#qdsp.plot()
feature_targets = qdsp.construct_feature_vector_matrix("HB_BUSAVG","A")

In [324]:
feature_targets

,Holiday,Hour,P(h-48),P(h-47),P(h-46),P(h-45),P(h-44),P(h-43),P(h-42),P(h-41),...,P(h-146),P(h-145),Day0,Day1,Day2,Day3,Day4,Day5,Day6,HB_BUSAVG
2012-01-09 00,0,0,13.49,13.10,12.76,12.93,13.42,16.65,23.36,23.97,...,25.18,24.30,1.0,0.0,0.0,0.0,0.0,0.0,0.0,14.59
2012-01-09 01,0,1,13.10,12.76,12.93,13.42,16.65,23.36,23.97,25.24,...,24.30,21.12,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.08
2012-01-09 02,0,2,12.76,12.93,13.42,16.65,23.36,23.97,25.24,26.87,...,21.12,19.97,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.06
2012-01-09 03,0,3,12.93,13.42,16.65,23.36,23.97,25.24,26.87,25.71,...,19.97,19.67,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.42
2012-01-09 04,0,4,13.42,16.65,23.36,23.97,25.24,26.87,25.71,25.39,...,19.67,18.59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,14.19
2012-01-09 05,0,5,16.65,23.36,23.97,25.24,26.87,25.71,25.39,25.05,...,18.59,20.10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,17.44
2012-01-09 06,0,6,23.36,23.97,25.24,26.87,25.71,25.39,25.05,24.51,...,20.10,26.19,1.0,0.0,0.0,0.0,0.0,0.0,0.0,27.67
2012-01-09 07,0,7,23.97,25.24,26.87,25.71,25.39,25.05,24.51,23.27,...,26.19,38.81,1.0,0.0,0.0,0.0,0.0,0.0,0.0,28.08
2012-01-09 08,0,8,25.24,26.87,25.71,25.39,25.05,24.51,23.27,23.80,...,38.81,44.83,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25.18
2012-01-09 09,0,9,26.87,25.71,25.39,25.05,24.51,23.27,23.80,25.02,...,44.83,31.78,1.0,0.0,0.0,0.0,0.0,0.0,0.0,26.33
